In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models-retrieval-with-hyperparameter-optimization/nvidia_logo.png" style="width: 90px; float: right;">

# Retrieval with hyperparameter optimization

## Overview

In this use case we will perform hyperparameter optimization using [Optuna](https://optuna.org/). Optuna is an open source hyperparameter optimization framework which automates hyperparameter search and can be used across a wide set of scenarios.

We will look at optimizing candidate retrieval on a dataset from a Kaggle competition, the [H&M Personalized Fashion Recommendations challenge](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations).

Hyperparameter optimization can be arbitrarily complex -- in this use case, we will look at optimizing the learning rate and embedding dimensionality to achieve best results on candidate generation. We will train a Matrix Factorization model on user-item pairs and maximize the hit rate at 100 (how many of top 100 retrieved candidates were indeed purchased).

## Why run an HPO (hyperparameter optimization) experiment in the first place?

Deep learninng models are highly sophisticated. Each architecture exposes many knobs that we can tweak. We can also optimizte the training procedure or the way in which we process data.

We usually have an idea of what a good range of values for any of these parameters would be, or at least we can make an educated guess. But it is impossible to pin point values that would give us adequate performance without ample experimentation.

We could perform all this work manually. We could train our model multiple times, tweaking the hyperparameters from run to run. We could track the results and systematically explore the parameter space.

This approach has its merits. We might learn about the effect each of the hyperparameter has and what they do in combination. We can learn a lot in this process.

But for day to day work, handing off all this effort to an HPO framework like Optuna can save us a lot of time. We no longer have to track the experiments since they all will be logged. We no longer have to babysit our model and rerun the training.

We write the code once, specify a range of hyperparameters for the experimentation framework to explore, and are off to the races. The experiment will run on its own and we can come back to it and review the results when it finishes.

### Learning objectives

- How to run a hyperoptimization experiment
- Candidate generation using Merlin Models

## Downloading and preparing the dataset

Let's begin by downloading the dataset. Please find the data on Kaggle [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data). We will only use `transactions_train.csv` which lists items that were purchased and maps the transactions to customers.

Please download the `transactions_train.csv` file and store it alongside the current notebook.

Let us read in the file and look at the data.

In [2]:
import os
os.chdir('../..')

# from merlin.datasets.synthetic import generate_data

# train_transformed = generate_data('user-item-interactions', 1000)
# valid_transformed = generate_data('user-item-interactions', 1000)

In [3]:
# from merlin.core.dispatch import get_lib
# import numpy as np

# transactions = get_lib().read_csv('transactions_train.csv', parse_dates=['t_dat'])
# transactions.head()

In [5]:
# from merlin.core.dispatch import get_lib
# import numpy as np

# transactions = get_lib().read_csv('examples/usecases/transactions_train.csv', parse_dates=['t_dat'])
# transactions.head()

Let's assign the last week to our validation set and treat the rest of that data as our train set. Additionally, let us remove purchases in our train set that were performed by customers who do not appear in our validation set. This will speed up our experiments (we have less data to train on) and in other experiments that I ran, including all the customers doesn't lead to better results.

In [2]:
seven_days_ago = transactions.t_dat.max() - np.timedelta64(7, 'D')

train_set = transactions[transactions.t_dat < seven_days_ago]
validation_set = transactions[transactions.t_dat >= seven_days_ago]

validation_set_customers = validation_set.customer_id.unique()
train_set = train_set[train_set.customer_id.isin(validation_set_customers)]

NameError: name 'transactions' is not defined

Before we can proceed with training we need to preprocess our data.

We will only use `customer_id` and `article_id` pairs. Still, a neural network expects them to be represented as continuous integers. We need to go from how they are represented in our dataset to that desired representation.

In order to do so, we will leverage `nvtabular` and the `Categorify` operator.

In [3]:
from nvtabular import *
from nvtabular import ops
from merlin.schema.tags import Tags
from merlin.models.tf.dataset import BatchedDataset

2022-10-26 02:13:02.073299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 02:13:02.073790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 02:13:02.073931: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


AttributeError: ID

We represent our data as `Merlin` `Datasets`.

In [ ]:
train_set = Dataset(train_set)
validation_set = Dataset(validation_set)

We now define the operations we want to apply to our data.

In [ ]:
customer_id = ['customer_id'] >> ops.Categorify() >> ops.TagAsUserID()
article_id = ['article_id'] >> ops.Categorify() >> ops.TagAsItemID()

And we proceed with fitting the workflow and transforming both the train and validation datasets.

In [ ]:
workflow = Workflow(customer_id + article_id)
train_transformed = workflow.fit_transform(train_set)

valid_transformed = workflow.transform(validation_set)

We are now ready to train our model and perform hyperparameter optimization.

## Hyperparameter optimization using optuna

We will train a retrieval model. Customers have performed a number of purchases in the last week of data that we are using as our validation set.

Our objective will be to train a retrieval model, Matrix Factorization, and to generate 100 candidates for each customer.

The goal is to maximize the count of purchases that appear among our candidates.

### The 3 components of hyperparameter optimization with Optuna

With Optuna, you create a `study`. A `study` is an optimization session with a number of trials. A `trial` is a single experiment, a call of the objective function.

A `parameter` is a variable whose value we will optimize.

This is a brief primer but should provide you with all the information necessary to get started with Optuna. You can find further information in Optuna's documentation [here](https://Optuna.readthedocs.io/en/stable/tutorial/index.html).

Below we will run an Optuna `study`. The `parameters` we will optimize are:

* embedding dimensionality
* learning rate
* number of epochs

Embedding dimensionality is how we control the capacity of the model. We would like to provide it with just the right amount of expressive power. If we endow our model with too great of a capacity, our model will overfit to our training data and it's ability to generalize to unseen data will suffer.

On the other hand, if we train too simple of a model, its performance will be limited as it will not be able to capture much of the signal in our train data.

The learning rate and number of epochs deal with the technical aspects of training our model. As the model might train differently depending on its capacity, we will perform a grid search across all these parameters in order to find a combination that will perform best.

### The setup of our study

Before we proceed further, please make sure you have `optuna` and `plotly` installed.

You can do so by executing the following line from your terminal:

`pip install optuna plotly`

In [3]:
from merlin.schema.tags import Tags

In [4]:
import merlin.models.tf as mm
# import tensorflow as tf
# import optuna

AttributeError: ID

Below we define an `objective` function. Running it is a single `trial` in our optimization `study`.

Each call of the `objective` function will train a model and evaluate its performance on the validation set. It will also return the count of candidates that were purchased across all the customers in the validation set. This is the value that we will strive to optimize.

In [ ]:
def calculate_hit_rate_at_100(model):
    item_features = train_transformed.schema.select_by_tag(Tags.ITEM_ID).column_names
    item_dataset = train_transformed.to_ddf()[item_features].drop_duplicates().compute()
    
    item_dataset = Dataset(item_dataset)
    top_k_rec = model.to_top_k_recommender(item_dataset, 100)
    
    users_schema = train_transformed.schema.select_by_tag(Tags.USER_ID)
    user_features = users_schema.column_names

    unique_users = train_transformed.to_ddf()[user_features].drop_duplicates().compute()
    users_dataset = Dataset(unique_users, schema=users_schema)
    _, cls_idxs = top_k_rec.predict(BatchedDataset(users_dataset, 100, shuffle=False, schema=users_schema))
    
    customer_id_mapping = get_lib().read_parquet('.//categories/unique.customer_id.parquet')
    customer_ids_preds = customer_id_mapping.to_pandas().customer_id.iloc[1:].values
    article_id_mapping = get_lib().read_parquet('.//categories/unique.article_id.parquet')
    
    validation_set_df = validation_set.compute()
    validation_set_df = validation_set_df.drop_duplicates(['customer_id', 'article_id'])
    val_cust2purchases = validation_set_df.to_pandas().groupby('customer_id')['article_id'].apply(list)
    
    id2a = article_id_mapping.to_pandas().article_id.to_dict()
    
    hit_rate = 0
    for i in range(cls_idxs.shape[0]):
        current_cust = customer_ids_preds[i]
        purchases = set(val_cust2purchases[current_cust])
        candidates = set(int(id2a[c]) for c in cls_idxs[i])
        hit_rate += len(purchases.intersection(candidates))
        
    return hit_rate

Below you will see me define `trial.suggest_float(...)` and `trial.suggest_int(...)`. In this notebook we will perform exhaustive grid search to explore the hyperparameter space. But Optuna comes with many more strategies for finding good hyperparameters.

Instead of trying all the values, we could have Optuna intelligently converge to a good set of hyperparameters without having to try out all of their combinations.

This is what the `trial.suggest...` syntax facilitates. Unfortunately, even if we want to do an exhaustive search over all the parameters, we stil need to follow Optuna's syntax and specify the permissible ranges a value can take.

If you are looking for a follow up activity to this notebook to learn more about HPO, consider replacing the `optuna.samplers.GridSampler` with another hyperparameter search strategy in one of the cells below. What results do you get? How many trials did you have to perform?

In [ ]:
def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1)
    num_epochs = trial.suggest_int('num_epochs', 1, 100)
    embedding_dim = trial.suggest_int('embedding_dim', 4, 128)
    
    model = mm.MatrixFactorizationModel(train_transformed.schema, embedding_dim)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0,
    )

    model.compile(optimizer, loss=loss)

    model.fit(train_transformed, validation_data=valid_transformed, batch_size=1024, epochs=num_epochs)
    
    return calculate_hit_rate_at_100(model)

For this example, we will use the `GridSampler`. We define parameter values we would like Optuna to perform an exhaustive search over in `search_space`.

Optuna features several other samplers that can navigate the search space using various algorithms. You can read about them in the documentation [here](https://optuna.readthedocs.io/en/stable/reference/samplers/index.html).

Still, exhaustive search is often the way to go. It guarantees the entire space will be explored and gives us an easy way to refine the experiments (for example, to focus on a specific area in the hyperparameter space we might want to learn more about).

In [ ]:
search_space = {
    'learning_rate': [5e-3, 1e-3, 1e-4],
    'num_epochs': [3, 6, 9],
    'embedding_dim': [16, 32, 48]
}

Our `study` will execute 27 `trials` (3 * 3 * 3) for us. I am capturing the output below as otherwise the notebook would become very hard to read (a lot information is produced during each training run).

We will use a `GridSampler` that will iterate over all the possible combinations of the hyperaparameters we chose for our `study`. This is just one of the many ways supported by `Optuna` for exploring the hyperparameter space. There are many sampler and sampling strategies one might chose from.

We set the `n_trials` parameter to 100. Still, `Optuna` will only run 27 runs and will terminate when it has tested all the possible hyperaparameter combinations.

In [ ]:
%%capture

study = optuna.create_study(sampler=optuna.samplers.GridSampler(search_space), direction='maximize')
study.optimize(objective, n_trials=100)

Now that we have performed hyperparameter optimization (a `study`), let's explore the results.

We can query the `study` for the best parameters along with the best attained result.

In [ ]:
study.best_params

The best attained result is displayed below. This is the count of candidates that were purchased in the validation week.

It is the value that we want to drive up as high as possible.

In [ ]:
study.best_value

We can also draw a parallel coordinate plot to take a look at a graphical depiction of the impact of the hyperparameters.

The vertical axis on the left gives us the values our `objective` function returned. Along the horizontal axis we have the parameters whose values we optimized.

These graphs are sometimes not easy to read, but here we can clearly see that higher values of the objective function are associated with greater embedding size and possibly lower learning rate (though that association seems to be less strong).

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

Contour plots can also often be helpful to understand better the interaction between two parameters.

Here we are looking at the learning rate and number of epochs. The medium learning rate value (`0.001` out of the 3 possible values we provided) along with longer training seems to perform best.

In [ ]:
optuna.visualization.plot_contour(study,params=['learning_rate','num_epochs'])

Another very useful functionality is hyperparameter importance. This information needs to be taken with a grain of salt, but can help us identify hyperparameters to focus on.

In [ ]:
optuna.visualization.plot_param_importances(study)

We can also take a look individually at the values our parameters took and the results we got.

It is interesting to note to what extent learning rate of `1e-3` outperforms the other learning rate values across a broad set of other parameters.

In [ ]:
optuna.visualization.plot_slice(study)

We can also query the `study` for the number of `trials` that have been run.

In [ ]:
len(study.trials)

Last but not least, we can take a look at each individual run as follows.

In [ ]:
study.trials[0]

We can access various pieces of information of each run for custom analysis.

In [ ]:
study.trials[0].params

In [ ]:
from collections import defaultdict

runtime_by_epoch_count = defaultdict(lambda: list())

for i in range(len(study.trials)):
    runtime_by_epoch_count[study.trials[i].params['num_epochs']].append(
        (study.trials[i].datetime_complete - study.trials[i].datetime_start).seconds
    )

for epoch_count in [3, 6, 9]:
    print(f'Mean runtime in seconds when training with {epoch_count} epochs: {np.mean(runtime_by_epoch_count[epoch_count]):.02f} seconds.')

## Summary

In this notebook, we have trained a retrieval model and optimized the parameters we used for training using `Optuna`. This approach offered a dramatic improvement in performance. Some hyperaparameter combinations resulted in our candidates containing as few as several hundred of purchased items.

With tuned hyperparameters we were able to retrieve 7238 candidates that overlap with items that were actually purchased.

`Optuna` is a powerful, open source hyperameter optimization framework that can be leveraged with minimum configuration to achieve good results. More elaborate strategies for searching the hyperparameter space can also be utilized, however performing an exhaustive grid search (how we did above) is a great approach to improving the performance of your model.

Other strategies of searching the hyperparameter space can be interesting to experiment with, can reduce the total runtime of a study, can offer greater flexibility, and certainly should be attempted so that you can get a feel for how they fit into your workflow, but might not necessarily lead to better results than exhaustive search.